In [1]:
#EDA - Exploratory data analysis

In [2]:

# EDA
# Feature Engineering - handle missing data
# Feature selection

In [3]:
transactions = [
    {"customer_id": 1, "date": "2023-01-01", "amount": 200},
    {"customer_id": 1, "date": "2023-01-10", "amount": 300},
    {"customer_id": 2, "date": "2023-01-05", "amount": 150},
    {"customer_id": 2, "date": "2023-01-20", "amount": 400},
    {"customer_id": 3, "date": "2023-01-03", "amount": 100}
]


In [4]:
transactions

[{'customer_id': 1, 'date': '2023-01-01', 'amount': 200},
 {'customer_id': 1, 'date': '2023-01-10', 'amount': 300},
 {'customer_id': 2, 'date': '2023-01-05', 'amount': 150},
 {'customer_id': 2, 'date': '2023-01-20', 'amount': 400},
 {'customer_id': 3, 'date': '2023-01-03', 'amount': 100}]

In [5]:
result = {}

for txn in transactions:
    cust_id = txn['customer_id']
    amount = txn['amount']
    
    if cust_id in result:
        result[cust_id] = result[cust_id]+amount
    else:
        result[cust_id] = amount

In [6]:
result

{1: 500, 2: 550, 3: 100}

In [14]:
def second_largest(nums):
    nums = [4, 5, 1, 2, 1, 4, 5, 3]
    second_largest = []

    for n in nums:
        if n not in second_largest:
            second_largest.append(n)
    return sorted(second_largest)[-2]

In [16]:
print(second_largest([32,3,45,678,87,645,356,78]))


4


In [42]:
# EDA - Exploratory Data Analysis - pandas practice codes
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv('./lpdg_test.csv')

In [44]:
df.head(3)

,user_id,age,signup_date,purchase_amount,country,device_type,traffic_source,session_duration_sec,pages_visited,is_repeat_user
0,101.0,25.0,2023-01-15,500.0,IN,mobile,organic,120,3,0
1,102.0,NaN,2023-02-10,1200.0,US,desktop,paid,300,6,1
2,103.0,35.0,2023-02-20,-100.0,IN,mobile,paid,45,1,0


In [45]:
df.isnull().sum()

user_id                 1
age                     3
signup_date             1
purchase_amount         1
country                 0
device_type             0
traffic_source          0
session_duration_sec    0
pages_visited           0
is_repeat_user          0
dtype: int64

In [46]:
df.columns

Index(['user_id', 'age', 'signup_date', 'purchase_amount', 'country',
       'device_type', 'traffic_source', 'session_duration_sec',
       'pages_visited', 'is_repeat_user'],
      dtype='object')

In [47]:
df.value_counts(["device_type"])

device_type
mobile         5
desktop        3
tablet         2
Name: count, dtype: int64

In [48]:
df.value_counts(["traffic_source"])

traffic_source
organic           4
paid              4
referral          2
Name: count, dtype: int64

In [75]:
# dropping the nan values
df_n=df.dropna(subset=["user_id"])
df=df.dropna(subset=["signup_date"])

In [83]:
# filling the nan values with meadium or avg or mean
df_n["age"] = df_n["age"].fillna(df_n["age"].median())

In [84]:
# coverting date to to datetime
df_n["signup_date"] = pd.to_datetime(df_n["signup_date"],errors="coerce")

In [85]:
#### filtering based  on condition --> this will create a new df in inplace is not used?
df = df[df["purchase_amount"]>=0]

In [86]:
# feature engineering the columns --> creating a new columns

user_spend = df_n.groupby("user_id")["purchase_amount"].sum().reset_index()

user_spend["customer_type"] = pd.cut(
    user_spend["purchase_amount"],
    bins =[-float('inf'),5000,10000,float('inf')],
    labels=['low','medium','high']
)
df_n =df_n.merge(
    user_spend[["user_id","customer_type"]],
    on = "user_id",
    how="left"
)



In [87]:
df_n.head(5)

,user_id,age,signup_date,purchase_amount,country,device_type,traffic_source,session_duration_sec,pages_visited,is_repeat_user,customer_type_x,signup_month,customer_type_y
0,101.0,25.0,2023-01-15,500.0,IN,mobile,organic,120,3,0,low,2023-01,low
1,102.0,27.0,2023-02-10,1200.0,US,desktop,paid,300,6,1,low,2023-02,low
2,104.0,45.0,NaT,7000.0,UK,desktop,organic,600,10,1,medium,NaT,medium
3,106.0,27.0,2023-03-15,15000.0,US,mobile,paid,900,15,1,high,2023-03,high
4,101.0,25.0,2023-01-20,800.0,IN,mobile,organic,180,5,1,low,2023-01,low


In [88]:
## Grouping and multiple aggregations

df_n["signup_month"] = df_n["signup_date"].dt.to_period("M")

summary = (
    df_n.groupby("signup_month").agg(
        total_revenue =("purchase_amount","sum"),
        unique_users = ("user_id","nunique"),
        avg_purchase = ("purchase_amount","mean")
    ).reset_index()
)

In [89]:
df_n

,user_id,age,signup_date,purchase_amount,country,device_type,traffic_source,session_duration_sec,pages_visited,is_repeat_user,customer_type_x,signup_month,customer_type_y
0,101.0,25.0,2023-01-15,500.0,IN,mobile,organic,120,3,0,low,2023-01,low
1,102.0,27.0,2023-02-10,1200.0,US,desktop,paid,300,6,1,low,2023-02,low
2,104.0,45.0,NaT,7000.0,UK,desktop,organic,600,10,1,medium,NaT,medium
3,106.0,27.0,2023-03-15,15000.0,US,mobile,paid,900,15,1,high,2023-03,high
4,101.0,25.0,2023-01-20,800.0,IN,mobile,organic,180,5,1,low,2023-01,low
5,103.0,27.0,NaT,4000.0,UK,tablet,organic,150,2,0,low,NaT,low
6,107.0,29.0,2023-03-30,200.0,IN,mobile,referral,60,1,0,low,2023-03,low


In [93]:
## why not to loop over in df  and preffer the vectorised operation as its fast and has low time complexity
# df["is_high"] = 0
# for i in range(len(df)):
#     if df.loc[i,"purchase_amount"] >1000:
#         df.loc[i,"is_high"] = 1



In [96]:
# Use this
df["is_high"] = (df(["purchase_amount"] > 10000).astype(int))

TypeError: '>' not supported between instances of 'list' and 'int'

In [90]:
#df.to_csv('./lpdg_test.csv', index=False)